In [6]:
from pyspark.sql import SparkSession

"""make an instance of a SparkSession called 'spark'."""
spark = SparkSession.builder.master('local').getOrCreate()


22/06/24 13:45:01 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/06/24 13:45:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/24 13:45:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [97]:
import os
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

# custom_schema = StructType([
#     StructField('index', IntegerType()),
#     StructField('artist_popularity', LongType()),
#     StructField('followers', LongType()),
#     StructField('genres', StringType()),
#     StructField('id', StringType()),
#     StructField('name', StringType()),
#     StructField('track_id', StringType()),
#     StructField('track_name_prev', StringType()),
#     StructField('type', StringType())])
# Read the Spotify artists CSV file into a Spark DataFrame
spark_df = (spark.read.format("csv"
                        ).options(header="true"
                        ).schema("index int, artist_popularity int, followers int, genres string, id string, name string, track_id string, track_name_prev string, type string"
                        ).load("data/spotify_artists.csv"))


In [98]:

spark_df.withColumnRenamed('', 'index').columns



['index',
 'artist_popularity',
 'followers',
 'genres',
 'id',
 'name',
 'track_id',
 'track_name_prev',
 'type']

In [99]:
"""Profile the Data:"""
# Show a description (summary) of the Spark DataFrame.
spark_df.describe


<bound method DataFrame.describe of DataFrame[index: int, artist_popularity: int, followers: int, genres: string, id: string, name: string, track_id: string, track_name_prev: string, type: string]>

In [100]:
spark_df.drop('index').columns


['artist_popularity',
 'followers',
 'genres',
 'id',
 'name',
 'track_id',
 'track_name_prev',
 'type']

In [101]:
# Print the schema of the DataFrame.
spark_df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- artist_popularity: integer (nullable = true)
 |-- followers: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name_prev: string (nullable = true)
 |-- type: string (nullable = true)



In [102]:
# Select and show just the first 10 values in the 'name' and 'genres' columns.
spark_df.select('name', 'genres').show(10)


+--------------------+--------------------+
|                name|              genres|
+--------------------+--------------------+
|       Juliano Cezar|['sertanejo', 'se...|
|      The Grenadines|                  []|
|             Gangway| ['danish pop rock']|
|               FADES|['uk alternative ...|
| Jean-Pierre Guignon|  ['french baroque']|
|              Filhos|                  []|
|                Eloq|                  []|
|              Fravær|                  []|
|       Camille Pépin|                  []|
|Pepe Willberg & T...|['classic finnish...|
+--------------------+--------------------+
only showing top 10 rows



In [103]:
from pyspark.sql.functions import regexp_replace
# Where the genre is an empty list, replace it with ['elevator music'].
spark_df = spark_df.withColumn('genres', regexp_replace(
    'genres', r"\[\]", "['elevator music']"))
spark_df.select('name', 'genres').show(10)


+--------------------+--------------------+
|                name|              genres|
+--------------------+--------------------+
|       Juliano Cezar|['sertanejo', 'se...|
|      The Grenadines|  ['elevator music']|
|             Gangway| ['danish pop rock']|
|               FADES|['uk alternative ...|
| Jean-Pierre Guignon|  ['french baroque']|
|              Filhos|  ['elevator music']|
|                Eloq|  ['elevator music']|
|              Fravær|  ['elevator music']|
|       Camille Pépin|  ['elevator music']|
|Pepe Willberg & T...|['classic finnish...|
+--------------------+--------------------+
only showing top 10 rows



In [ ]:
# For the columns 'artist_popularity' and 'followers', cast the data type as integers.
"""done in the schema"""

In [104]:
# Sort the data in descending order by number of followers.

spark_df.select('*').sort('followers', ascending=False).show(5)


22/06/24 14:50:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , artist_popularity, followers, genres, id, name, track_id, track_name_prev, type
 Schema: index, artist_popularity, followers, genres, id, name, track_id, track_name_prev, type
Expected: index but found: 
CSV file: file:///home/fossa/deb/tests/spark_fun/data/spotify_artists.csv


+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+
|index|artist_popularity|followers|              genres|                  id|         name|            track_id|track_name_prev|  type|
+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+
|55251|               92| 41561693|   ['pop', 'uk pop']|6eUKZXaKkcviH0Ku9...|   Ed Sheeran|7qiZfU4dY1lWllzX7...|       track_35|artist|
|53392|               98| 34680740|['canadian hip ho...|3TVXtAsR1Inumwj47...|        Drake|116H0KvKr2Zl4RPuV...|       track_71|artist|
|52620|               90| 30560149|['dance pop', 'po...|5pKCCKE2ajJHZ9KAi...|      Rihanna|2Ce5IyMlVRVvN997Z...|       track_38|artist|
|54447|               88| 26824224|['canadian pop', ...|1uNFoZAHBGtllmzzn...|Justin Bieber|3A7qX2QjDlPnazUsR...|        track_2|artist|
|42872|              100| 26309771|['dance pop',

In [ ]:
# 'artist_popularity' is a rank out of 100. Write a user defined function that will divide each popularity value by 100. Rename the column 'popularity_percent'.
